## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/22/16,Citibike to Work,1.45,NaN,10.0,0.17,8.70,Citibike
1,10/19/16,Citibike to Work,1.53,NaN,10.0,0.17,9.18,Citibike
2,11/4/16,Citibike to Work,1.12,NaN,10.0,0.17,6.72,Citibike
3,9/17/20,Wards Island,12.70,1.0,33.0,1.55,8.19,Specialized
4,4/14/21,Dn ES Up LWS,15.84,1.0,55.0,1.92,8.26,Specialized
...,...,...,...,...,...,...,...,...
368,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
369,6/16/21,Up Dn WS to GWB Up ES,30.18,3.0,32.0,3.53,8.54,Specialized
370,6/3/20,To From Economy Candy,8.08,1.0,10.0,1.17,6.93,Specialized
371,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/22/16,Citibike to Work,1.45,0.0,10.0,0.17,8.70,Citibike
1,10/19/16,Citibike to Work,1.53,0.0,10.0,0.17,9.18,Citibike
2,11/4/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike
3,9/17/20,Wards Island,12.70,1.0,33.0,1.55,8.19,Specialized
4,4/14/21,Dn ES Up LWS,15.84,1.0,55.0,1.92,8.26,Specialized
...,...,...,...,...,...,...,...,...
368,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
369,6/16/21,Up Dn WS to GWB Up ES,30.18,3.0,32.0,3.53,8.54,Specialized
370,6/3/20,To From Economy Candy,8.08,1.0,10.0,1.17,6.93,Specialized
371,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized


In [6]:
# Turning Index into Orig_Index Column to trace back to original records
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,6/22/16,Citibike to Work,1.45,0.0,10.0,0.17,8.70,Citibike
1,1,10/19/16,Citibike to Work,1.53,0.0,10.0,0.17,9.18,Citibike
2,2,11/4/16,Citibike to Work,1.12,0.0,10.0,0.17,6.72,Citibike
3,3,9/17/20,Wards Island,12.70,1.0,33.0,1.55,8.19,Specialized
4,4,4/14/21,Dn ES Up LWS,15.84,1.0,55.0,1.92,8.26,Specialized
...,...,...,...,...,...,...,...,...,...
368,368,5/26/20,Down Up LWS,13.35,1.0,30.0,1.50,8.90,Specialized
369,369,6/16/21,Up Dn WS to GWB Up ES,30.18,3.0,32.0,3.53,8.54,Specialized
370,370,6/3/20,To From Economy Candy,8.08,1.0,10.0,1.17,6.93,Specialized
371,371,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [9]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,1.45,0.17,8.70,Citibike
1,1.53,0.17,9.18,Citibike
2,1.12,0.17,6.72,Citibike
3,12.70,1.55,8.19,Specialized
4,15.84,1.92,8.26,Specialized
...,...,...,...,...
368,13.35,1.50,8.90,Specialized
369,30.18,3.53,8.54,Specialized
370,8.08,1.17,6.93,Specialized
371,17.67,2.08,8.48,Specialized


In [10]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [11]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [12]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [13]:
# Checking training data shape
X_train.shape

(279, 3)

In [14]:
# Checking testing data shape
X_test.shape

(94, 3)

In [15]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [16]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [17]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [18]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9787234042553191


In [19]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Specialized
2,Specialized
3,Specialized
4,Citibike
...,...
89,Specialized
90,Specialized
91,Citibike
92,Citibike


In [20]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
278,Specialized
91,Specialized
126,Specialized
60,Specialized
280,Citibike
...,...
316,Specialized
121,Specialized
56,Citibike
131,Citibike


In [21]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,278,Specialized
1,91,Specialized
2,126,Specialized
3,60,Specialized
4,280,Citibike
...,...,...
89,316,Specialized
90,121,Specialized
91,56,Citibike
92,131,Citibike


In [22]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,278,Specialized,Specialized
1,91,Specialized,Specialized
2,126,Specialized,Specialized
3,60,Specialized,Specialized
4,280,Citibike,Citibike
...,...,...,...
89,316,Specialized,Specialized
90,121,Specialized,Specialized
91,56,Citibike,Citibike
92,131,Citibike,Citibike


In [46]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
22,277,Specialized,Citibike
37,312,Specialized,Citibike


In [47]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,277,Specialized,Citibike,6/28/20,Uptown Trio,3.95,0.0,26.0,0.43,9.12,Specialized
1,312,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54,Specialized


In [48]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,277,Specialized,Citibike,6/28/20,Uptown Trio,3.95,0.0,26.0,0.43,9.12
1,312,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54
